## Advanced example

A more advanced example is provided below, for designing a simple leakage detection algorithm for the `Net2` benchmark network. The goal is to generate pressure bounds (i.e., the adaptive upper and lower levels of pressure expected at a node, given the uncertainty in model parameters) which can be used to detect events in the system, e.g., by comparing them with available pressure sensor measurements.

### Initialize EPANET Python Toolkit (EPyT)

You should always begin with this command to import the toolkit.

[EPyT](https://github.com/OpenWaterAnalytics/EPyT) is available on [PyPI](https://pypi.org/project/epyt/) and can be installed via `pip install epyt`. To upgrade to the latest version if it's already installed, use `pip install --upgrade epyt`.

In [ ]:
%pip install epyt

In [ ]:
from epyt import epanet
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Create a function to run the simulation and return the pressure results
def compute_bounds(G, nsim, base_demands, eta_bar, node_index):
    # Seed number to always get the same random results
    np.random.seed(1)
    # Initialize matrix to save MCS pressures
    pmcs = [None for _ in range(nsim)]
    for i in range(nsim):
        # Compute new base demands
        delta_bd = (2 * np.random.rand(1, len(base_demands))[0] - 1) * eta_bar * base_demands
        new_base_demands = base_demands + delta_bd
        
        # Set base demands
        G.setNodeBaseDemands(new_base_demands)
        
        # Compute pressures at each node
        pmcs[i] = G.getComputedHydraulicTimeSeries().Pressure
        print(f"Epoch {i}")

    # Compute upper and lower bounds
    pmulti = []
    for i in range(nsim):
        pmulti.append(pmcs[i][:, node_index - 1])
    pmulti = np.vstack(pmulti)
    ub = np.max(pmulti, axis=0)
    lb = np.min(pmulti, axis=0)
    meanb = np.mean(pmulti, axis=0)

    return pmulti, ub, lb, meanb

For generating leakage events, it’s useful to activate the Pressure-Driven Analysis (PDA), instead of using the default Demand-Driven Analysis (DDA), as the effect on demands due to pressure drops during leakages is not negligible. Moreover, PDA avoids simulation errors due to negative pressures.

In [ ]:
def activate_PDA(G):
    type = 'PDA'
    pmin = 0
    preq = 0.1
    pexp = 0.5
    G.setDemandModel(type, pmin, preq, pexp)  # Sets the demand model


In [ ]:
if __name__ == "__main__":

    # Prepare network for Monte Carlo Simulations
    # Load network
    inp_name = 'Net2.inp'  # 'L-TOWN.inp'
    G = epanet(inp_name)
    # Pressure driven analysis
    activate_PDA(G)

In [ ]:
    # Get nominal base demands
    base_demands = G.getNodeBaseDemands()[1]
    print(base_demands)

We assume we have a pressure sensor at the node with ID “11”. We will now create the pressure bounds at that node, using Monte Carlo Simulations (MCS). We assume that there is 2% uncertainty in the nominal base demands compared to the actual demand, which is evenly distributed with the nominal value as the mean. We consider a suitable number of MCS (we use 100 epochs for computational convenience, however, more simulations would provide a more accurate estimation of the bounds). Starting from the current time, we run the simulations for 56 hours for each randomized scenario, the computed pressure measurements are recorded.

In [ ]:
    # Number of simulations
    nsim = 100
    # Pressure Simulations at Node 5
    node_id = '11'
    node_index = G.getNodeIndex(node_id)
    # 5% max uncertainty in base demands
    eta_bar = 0.02
    pmulti, ub, lb, meanb = compute_bounds(G, nsim, base_demands, eta_bar, node_index)
    print(pmulti, ub, lb, meanb)

The upper and lower bounds can be computed by processing all the simulated pressure measurements using numpy methods. The results are depicted in `Figure 3.` Given a sufficient number of simulations, we expect that under normal conditions, pressure at node `“11”` will reside between those bounds. In blue, the average pressure computed by the MCS is depicted.

In [ ]:
    # Plots
    pressure_units = G.units.NodePressureUnits
    plt.rc('xtick', labelsize=7)
    plt.rc('ytick', labelsize=7)
    fig, ax = plt.subplots(figsize=(4, 3))
    ax.plot(ub, 'k')
    ax.plot(lb, 'k')
    ax.plot(meanb, 'b')
    ax.grid(True)
    ax.legend(['Upper bound', 'Lower bound', 'Average'], loc='upper right', fontsize=7)
    ax.set_title(f'Pressure bounds, Node ID: {node_id}', fontsize=8)
    ax.set_xlabel('Time (hours)', fontsize=7)
    ax.set_ylabel(f'Pressure ({pressure_units})', fontsize=7)
    plt.show()
    # fig.savefig('figures/paper_pressure_bounds.png', dpi=300)

To demonstrate the detection ability of the proposed approach, we simulate a leakage with 50 gallons per minute (GPM) outflow at the node with ID `“7”`, starting `20 hours` after the current time. During a leakage event, we expect that the pressure will drop, and for a sufficiently large leak, the measured pressure can fall below the estimated lower bound, thus triggering a leakage warning.


In [ ]:
    # Add leakage at Node ID 7 after 20 hours
    leak_scenario = 50
    leak_start = 20
    leak_value = 50  # GPM unit
    leak_node_id = '7'
    leak_node_index = G.getNodeIndex(leak_node_id)
    leak_pattern = np.zeros(max(G.getPatternLengths()))
    leak_pattern[leak_start:] = 1
    pattern_index = G.addPattern('leak', leak_pattern)
    G.setNodeDemandPatternIndex(leak_node_index, pattern_index)
    G.setNodeBaseDemands(leak_node_index, leak_value)

In [ ]:
    # Compute pressures
    scada_pressures = G.getComputedHydraulicTimeSeries().Pressure

The detection algorithm compares the lower pressure bound of node `“7”` with the actual pressure as follows:

In [ ]:
    p7 = scada_pressures[:, node_index-1]
    e = p7 - lb
    alert = e < 0
    detectionTime = np.argmax(alert>1)

In [ ]:
    # Bounds with Leakage
    fig, ax = plt.subplots(figsize=(4, 3))
    ax.plot(ub, 'k')
    ax.plot(lb, 'k')
    ax.plot(p7, 'r')
    ax.grid(True)
    ax.legend(['Upper bound', 'Lower bound', 'Sensor'], loc='upper right', fontsize=7)
    ax.set_title(f'Pressure bounds, Leak Node ID: {leak_node_id}', fontsize=8)
    ax.set_xlabel('Time (hours)', fontsize=7)
    ax.set_ylabel(f'Pressure ({pressure_units})', fontsize=7)
    plt.show()
    # fig.savefig('figures/paper_pressure_bounds_leak.png', dpi=300)

We observe that in this use case, until time `27 hours`, the sensor measurement was within the upper and lower bounds computed in the previous step, therefore there was a `7 hour delay` in detecting the leakage.

In [ ]:
    # Leakage alert
    fig, ax = plt.subplots(figsize=(4, 3))
    ax.plot(alert)
    ax.set_title(f'Leakage alert', fontsize=8)
    ax.set_xlabel('Time (hours)', fontsize=7)
    plt.show()
    # fig.savefig('figures/paper_leakage_alert.png', dpi=300)